In [ ]:
import requests
import json
import pandas as pd

# Make a request to the JSON endpoint and retrieve the data
alldata=[]
for i in range(1,24):
        
    response = requests.get(f'https://www.boysnextdoor-apparel.co/collections/all/products.json?page={i}')
    data = json.loads(response.text)

    alldata.extend(data['products'])


df = pd.json_normalize(alldata)

df

,id,title,handle,body_html,published_at,created_at,updated_at,vendor,product_type,tags,variants,images,options
0,5751418519702,A.P.C. Daniela Logo-Print Denim Tote Bag,a-p-c-daniela-logo-print-denim-tote-bag,"<p><meta charset=""utf-8""><span data-mce-fragme...",2020-09-22T18:26:15+08:00,2020-09-22T18:26:16+08:00,2023-03-10T02:06:10+08:00,APC,Bag,"[Accessories, APC, Bag, Cotton, Denim, Navy, T...","[{'id': 36425838559382, 'title': 'Default Titl...","[{'id': 19781986779286, 'created_at': '2020-11...","[{'name': 'Title', 'position': 1, 'values': ['..."
1,7728812490978,and Wander Dyneema Stuffsack,and-wander-dyneema-stuffsack,"<p><span data-mce-fragment=""1"">A lightweight a...",2022-05-16T18:54:22+08:00,2022-05-16T18:54:22+08:00,2022-11-30T19:15:50+08:00,and Wander,Bag,"[and Wander, Bag, spo-disabled, White]","[{'id': 42831058600162, 'title': 'Default Titl...","[{'id': 37319399014626, 'created_at': '2022-05...","[{'name': 'Title', 'position': 1, 'values': ['..."
2,7696930504930,and Wander Heather Waist Bag Charcoal,and-wander-heather-waist-bag-charcoal,"<p><meta charset=""utf-8""><meta charset=""UTF-8""...",2022-04-21T23:27:44+08:00,2022-04-21T23:27:44+08:00,2022-11-30T19:15:50+08:00,and Wander,Bag,"[and Wander, Bag, Charcoal, Grey, spo-disabled...","[{'id': 42746265075938, 'title': 'Charcoal', '...","[{'id': 37179186413794, 'created_at': '2022-04...","[{'name': 'Color', 'position': 1, 'values': ['..."
3,7696940531938,and Wander Heather Waist Bag Navy,and-wander-heather-waist-bag-navy,"<p><meta charset=""utf-8""><meta charset=""UTF-8""...",2022-04-21T23:50:41+08:00,2022-04-21T23:50:41+08:00,2022-11-30T19:15:50+08:00,and Wander,Bag,"[and Wander, Bag, Charcoal, Grey, spo-disabled...","[{'id': 42746271760610, 'title': 'Charcoal', '...","[{'id': 37179207352546, 'created_at': '2022-04...","[{'name': 'Color', 'position': 1, 'values': ['..."
4,7767737467106,Axel Arigato Marathon Sneaker White Silver,axel-arigato-marathon-sneaker-white-silver,"<meta charset=""utf-8""><meta charset=""utf-8"">\n...",2022-06-15T15:57:15+08:00,2022-06-15T15:57:15+08:00,2022-11-30T19:15:52+08:00,Axel Arigato,Shoes,"[Axel Arigato, Leather, Marathon, Shoes, Train...","[{'id': 42992708190434, 'title': '39', 'option...","[{'id': 37520100655330, 'created_at': '2022-06...","[{'name': 'Size', 'position': 1, 'values': ['3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,7728767860962,Wood Wood Low Profile Mondana Cap,wood-wood-low-profile-mondana-cap,"<meta charset=""utf-8"">\n<p>Inspired by the lif...",2022-05-16T17:53:20+08:00,2022-05-16T17:53:20+08:00,2022-07-27T00:00:01+08:00,Wood Wood,Cap,"[Accessories, Cap, spo-disabled, Wood Wood]","[{'id': 42830937260258, 'title': 'Default Titl...","[{'id': 37319040073954, 'created_at': '2022-05...","[{'name': 'Title', 'position': 1, 'values': ['..."
666,7693151174882,WYM Heavy Weight Basic Big-Tee Beige,wym-heavy-weight-basic-big-tee-beige,"<meta charset=""utf-8"">\n<ul>\n<li>6 oz. Fabric...",2022-04-20T16:07:10+08:00,2022-04-20T16:07:10+08:00,2022-07-26T23:59:58+08:00,WYM,T-Shirt,"[Beige, Plain, spo-default, spo-disabled, T-Sh...","[{'id': 42738924912866, 'title': 'S', 'option1...","[{'id': 37167246835938, 'created_at': '2022-04...","[{'name': 'Size', 'position': 1, 'values': ['S..."
667,7693155107042,WYM Heavy Weight Basic Big-Tee Blue,wym-heavy-weight-basic-big-tee-blue,"<meta charset=""utf-8"">\n<ul>\n<li>6 oz. Fabric...",2022-04-20T16:08:32+08:00,2022-04-20T16:08:32+08:00,2022-07-26T23:59:58+08:00,WYM,T-Shirt,"[Blue, Plain, spo-default, spo-disabled, T-Shi...","[{'id': 42738934972642, 'title': 'S', 'option1...","[{'id': 37167260205282, 'created_at': '2022-04...","[{'name': 'Size', 'position': 1, 'values': ['S..."
668,7693148520674,WYM Heavy Weight Basic Big-Tee Grey,wym-heavy-weight-basic-big-tee-grey,"<meta charset=""utf-8"">\n<ul>\n<li>6 oz. Fabric...",2022-04-20T16:06:09+08:00,2022-04-20T16:06:09+08:00,2023-01-02T19:45:35+08:00,WYM,T-Shirt,"[Grey, Plain, spo-default, spo-disabled, T-Shi...","[{'id': 42

In [2]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Make a request to the JSON endpoint to retrieve the data
alldata=[]
for i in range(1,24):
        
    response = requests.get(f'https://www.boysnextdoor-apparel.co/collections/all/products.json?page={i}')
    data = json.loads(response.text)
    alldata.extend(data["products"])

# Create a pandas DataFrame from the JSON data
df = pd.json_normalize(alldata)

# Preprocessing the data
#we are making one feature of all the imp features and converting it in unicode
df['all_text'] = df['title'] + ' ' + df['body_html'] + ' ' + df['vendor'] + ' ' + df['product_type']
vectorizer = CountVectorizer(stop_words='english')
vectorized = vectorizer.fit_transform(df['all_text'].values.astype('U'))
similarity_matrix = cosine_similarity(vectorized)

# Find the alternates of the same product in the store
alternate_groups = []
for i in range(len(df)):
    alternate_group = [df['handle'][i]]
    for j in range(i+1, len(df)):
        if similarity_matrix[i][j] >= 0.9:
            alternate_group.append(df['handle'][j])
    if len(alternate_group) > 1:
        alternate_groups.append(alternate_group)

# Print the groups of alternate products
# print(alternate_groups)
alternate_groups
#to store we are creating a array
result=[]
for singlealternate in alternate_groups:
    singleresult = {"product alternates":[]}
    for productname in singlealternate:
        link = f"https://www.boysnextdoor-apparel.co/products/{productname}"
        singleresult["product alternates"].append(link)
    result.append(singleresult)

print(result)

[{'product alternates': ['https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-charcoal', 'https://www.boysnextdoor-apparel.co/products/and-wander-heather-waist-bag-navy']}, {'product alternates': ['https://www.boysnextdoor-apparel.co/products/beams-two-pocket-sweater-navy', 'https://www.boysnextdoor-apparel.co/products/beams-two-pocket-sweater-white']}, {'product alternates': ['https://www.boysnextdoor-apparel.co/products/beams-x-abu-garcia-limited-edition-shorts-black', 'https://www.boysnextdoor-apparel.co/products/beams-x-abu-garcia-limited-edition-shorts-green']}, {'product alternates': ['https://www.boysnextdoor-apparel.co/products/beams-x-columbia-limited-edition-shorts-black', 'https://www.boysnextdoor-apparel.co/products/copy-of-beams-x-columbia-limited-edition-shorts-navy', 'https://www.boysnextdoor-apparel.co/products/beams-x-columbia-logriver-bms-jacket-black', 'https://www.boysnextdoor-apparel.co/products/beams-x-columbia-logriver-bms-jacket-navy', 'htt